In [ ]:
pip install pystarburst

In [ ]:
import trino
from pystarburst import Session
import urllib3

urllib3.disable_warnings()

db_parameters = {
    "host": "parvathigokavarapu-s3trngcluster.trino.galaxy.starburst.io",
    "port": 443,
    "http_scheme": "https",
    "verify":False, #to skip the SSL verification
    # Setup authentication through login or password or any other supported authentication methods
    # See docs: https://github.com/trinodb/trino-python-client#authentication-mechanisms
    "auth": trino.auth.BasicAuthentication("parvathigokavarapu89@gmail.com/accountadmin", "")
}
try:
    session = Session.builder.configs(db_parameters).create()
    session.sql("select 1 as b").collect()
    print('cluster got connected')
    dfprod = session.sql("select *from s3datalakecatalog.discovered_schema.dell")
    dfprod.show()
except Exception as e:
    print(e)

In [ ]:
dfprod2=dfprod.rename("product type", "product_type")

### groupby products

In [62]:

dfprod3=dfprod2.group_by('product_type').count()\
    .rename('count', 'No_of_products')

In [ ]:
session.sql("create schema if not exists s3datalakecatalog.s3dataprdschema").collect()

In [ ]:
session.sql("drop table if exists s3datalakecatalog.s3dataprdschema.prdsupplier").collect()

In [ ]:

dfprod3.write.save_as_table("s3datalakecatalog.s3dataprdschema.prdsupplier")

In [ ]:
session.sql("select * from s3datalakecatalog.s3dataprdschema.prdsupplier").show()

In [65]:
print(dfprod3.schema.json())

{"type": "StructType", "fields": [{"column_identifier": {"normalized_name": "\"product_type\""}, "datatype": {"size": null, "type": "StringType"}, "nullable": true}, {"column_identifier": {"normalized_name": "\"no_of_products\""}, "datatype": {"type": "LongType"}, "nullable": true}]}


In [ ]:
def conver_sb_to_pd(dfsp):
    return pd.DataFrame(dfsp.collect(), columns=dfsp.columns)
    

In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
dffinal=conver_sb_to_pd(dfprod3)
dffinal.plot(kind='pie', figsize=(12,10),y='no_of_products',labels=dffinal['product_type'],legend=False)
plt.show()

In [ ]:

pip install pyspark